這邊我會假設你已經看過[Darknet-53_structure](https://github.com/shaoeChen/deeplearning/blob/master/tf2/Arch_YOLOv3_1_Darknet-53_structure.ipynb)與[YOLOv3_structure](https://github.com/shaoeChen/deeplearning/blob/master/tf2/Arch_YOLOv3_2_YOLOv3_structure.ipynb)的說明，而且你也已經完全明白模型的架構是怎麼一回事。

下面先給出眾多的參考資料，瞭解一個架構不容易，參考的自然也多了，正確的寫出引用、參照是學習過程中的必要：

* [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/)
* [pjreddie/darknet](https://github.com/pjreddie/darknet)
* [YOLOv3_論文翻譯連結](https://hackmd.io/@shaoeChen/SyjI6W2zB/https%3A%2F%2Fhackmd.io%2F%40shaoeChen%2FryHg904h9)
* [YOLOv3深度解析](https://blog.csdn.net/leviopku/article/details/82660381)
* [qqwweee/keras-yolo3](https://github.com/qqwweee/keras-yolo3)
* [YunYang1994/tensorflow-yolov3](https://github.com/YunYang1994/tensorflow-yolov3)
* [joymyhome_Yolov3 config file中pad的理解](https://blog.csdn.net/joymyhome/article/details/106349084)
* [華為雲開發者社區](https://segmentfault.com/a/1190000039009074)
* [pylessons_YOLOv3-TF2-mnist](https://pylessons.com/YOLOv3-TF2-mnist)

相關前置資料的處理可以參考另作[Arch_YOLOv2_dataset_preprocess.ipynb](https://github.com/shaoeChen/deeplearning/blob/master/tf2/Arch_YOLOv2_dataset_preprocess.ipynb)

我的docker上執行的版本為tensorflow 2.1，雖然現在流行人生苦短我用PyTorch，不過我還是先繼續tf + keras。

In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

指定使用的gpu

In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')

In [3]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

我們已經可以成功的從模型中取得粗、中、細的預測資訊，論文中我們知道，每個NxN的grid cell裡面都會有三個預測框的資訊。但不同尺度的預測框大小是不一樣的，這部份是必需要注意的，這從cfg檔中也可以看的出來。

9個框的大小如下：10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326  

粗(13x13)：116,90,  156,198,  373,326  
中(26x26)：30,61,  62,45,  59,119  
細(52x52)：10,13,  16,30,  33,23  

每一組數值代表的就是高跟寬，下面我們也整理一下這九個anchor box

In [4]:
import numpy as np 
# 1.25,1.625, 2.0,3.75, 4.125,2.875, 1.875,3.8125, 3.875,2.8125, 3.6875,7.4375, 3.625,2.8125, 4.875,6.1875, 11.65625,10.1875
# [8, 16, 32]
anchors='10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326'
anchors = [float(x) for x in anchors.split(',')]
anchors = np.array(anchors).reshape((3, 3, 2))
anchors

array([[[ 10.,  13.],
        [ 16.,  30.],
        [ 33.,  23.]],

       [[ 30.,  61.],
        [ 62.,  45.],
        [ 59., 119.]],

       [[116.,  90.],
        [156., 198.],
        [373., 326.]]])

In [5]:
ANCHORS = anchors.copy()

更多關於YOLO的相關資訊建議可以看一下[吳恩達老師的課程](https://hackmd.io/@shaoeChen/HJUZTKMZz/https%3A%2F%2Fhackmd.io%2Fs%2FSJXmp66KG)，連結是在下的讀書筆記，這可以幫助我們快速有一個概略的觀念。

首先，我們已經有模型的output，三組output的每個grid cell各有三個框的資訊，每個框裡面則有(1(有沒有框到) + 4(框的資訊) + 80(類別資訊))的資訊。如果我們把模型的output視為一種encoder，那就會需要一個decoder來處理模型的輸出。

框的資訊我們可以參考論文的一張圖：

<img src="https://hackmd.io/_uploads/BJlQ-JO3q.png" width="25%" />

框的資訊裡面包含它的中心點x、y以及寬(w)、高(h)，中心點的部份是利用sigmoid的特性(輸出為0~1之間的值)計算而得，而$C_x, C_y$的部份則是grid cell的索引，上圖為例，$C_x, C_y$為1, 1，這樣我們就知道是第1個row、第一個col的這個格式的中心點的output結果。

程式碼的部份保證參考[YunYang1994](https://github.com/YunYang1994/TensorFlow2.0-Examples/blob/master/4-Object_Detection/YOLOV3/core/yolov3.py)，只是加入自己的理解說明。這邊主要處理的就是將模型的輸出轉為我們需要的樣子。

In [23]:
def decoder(yolo_output, anchor_size: int = 0):
    """整理YOLOv3的output
            
    yolo output有三種：Mx13x13x255、Mx26x26x255、Mx52x52x255，其中M代表你的batch size，
        每個255裡面都有3個anchor boxes、每個anchor box都有1個置信度、4個框資訊以及80個類別預測結果
    
    anchor_size: NxN的N
        2: 13x13
        1: 26x26
        0: 52x52
    
    裡面關於anchor box數量、類別數量都是可以依你自己專案實際需求來調整的，
    不過這邊就不設置過多的參數來做這一塊，避免複雜化我們的說明過程。
    
    tf.tile，函數如其名，把你的input看成一個瓦片，然後利用tile複製看你要幾片，假設input=3x3
        (1, 2)，就是往右多貼一片，輸出為3x6
        (2, 1)，就是往下多貼一片，輸出為6x3
    """
    output_shape = yolo_output.shape
    # 因為我們在定義模型的時候並不確定它的batch_size為何，因此會是None
    # 如果這邊我們利用tensor.shape來取得靜態維度的話，那就會是None，後續就會報錯
    # 因此採用tf.shape取得動態維度，就可以正常執行
    batch_size = tf.shape(yolo_output)[0]
    grid_cell_nums = output_shape[1]
    
    # 這邊主要要將這個輸出轉成M x N x N x 3 x (5 + 80)，其中N指的是你將切片分割成N x N
    output_reshape = tf.reshape(tensor=yolo_output, shape=(batch_size, grid_cell_nums, grid_cell_nums, 3, 85))
    print(f'output_reshape: {output_reshape.shape}')
    
    # 框的中心點
    output_reshape_txty = output_reshape[:, :, :, :, 0: 2]
    print(f'output_reshape_txty: {output_reshape_txty.shape}')
    # 框的寬高
    output_reshape_pwph = output_reshape[:, :, :, :, 2: 4]
    print(f'output_reshape_pwph: {output_reshape_pwph.shape}')
    # 有沒有東西的置信度
    output_reshape_conf = output_reshape[:, :, :, :, 4: 5]
    print(f'output_reshape_conf: {output_reshape_conf.shape}')
    # 類別
    output_reshape_class_prob = output_reshape[:, :, :, :, 5: ]
    print(f'output_reshape_class_prob: {output_reshape_class_prob.shape}')
    
    # 關於tile之後的結果我放在最下面附錄，方便直觀瞭解這邊的操作
    # tf.range之後的格式是秩，也就是(13, )，所以加入一個軸讓它變成(13, 1)，再利用tile來做複製    
    y = tf.tile(tf.range(grid_cell_nums, dtype=tf.int32)[:, tf.newaxis], [1, grid_cell_nums])
    print(f'y: {y.shape}')
    # tf.range之後的格式是秩，也就是(13, )，所以加入一個軸讓它變成(1, 13)，再利用tile來做複製    
    x = tf.tile(tf.range(grid_cell_nums, dtype=tf.int32)[tf.newaxis, :], [grid_cell_nums, 1])
    print(f'x: {x.shape}')
    # 再推一個軸，然後把兩個tensor沿著新增的軸疊起來
    # 這時候就變成[[0, 0], [1, 0], [2, 0], ...]
    # 如果是13x13的tensor，那這時候的維度就會是13x13x2
    xy_grid = tf.concat([x[:, :, tf.newaxis], y[:, :, tf.newaxis]], axis=-1)
    print(f'xy_grid: {xy_grid.shape}')
    # 這邊再推新軸，這時候就會是1x13x13x1x2
    # 然後再經過tile處理，這時候的維度就會是batch_size x 13 x 13 x 3 x 2
    # 這時候的維度就跟上面output_reshape_txty、output_reshape_pwph的維度一樣
    xy_grid = tf.tile(xy_grid[tf.newaxis, :, :, tf.newaxis, :], [batch_size, 1, 1, 3, 1])
    print(f'xy_grid: {xy_grid.shape}')
    # 將型別從int32轉float32
    xy_grid = tf.cast(xy_grid, tf.float32) 
    print(f'xy_grid: {xy_grid.shape}')    
    # 這是根據上圖的公式計算中心點
    # 因為我們上面已經整理一下大表，維度都一樣，所以只要直接加上xy_grid就可以得到中心點
    pred_xy = (tf.sigmoid(output_reshape_txty) + xy_grid)
    print(f'pred_xy: {pred_xy.shape}')
    # 這是根據上圖的公式計算寬高
    pred_wh = (tf.exp(output_reshape_pwph) * ANCHORS[anchor_size])
    print(f'anchor box: {ANCHORS[anchor_size]}')
    print(f'pred_wh: {pred_wh.shape}')
    # 把得到的結果堆疊起來，這時候的維度為batch size x 13 x 13 x 3 x 4 -> x、y、寬、高
    pred_xywh = tf.concat([pred_xy, pred_wh], axis=-1)
    print(f'pred_xywh: {pred_xywh.shape}')
    # 這個框有沒有東西的置信度
    pred_conf = tf.sigmoid(output_reshape_conf)
    print(f'pred_conf: {pred_conf.shape}')
    # 類別機率計算
    pred_prob = tf.sigmoid(output_reshape_class_prob)
    print(f'pred_prob: {pred_prob.shape}')
    
    return tf.concat([pred_xywh, pred_conf, pred_prob], axis=-1) 

弄個隨機張量，順便觀察過程中的維度變化，比較清楚到底我們自己在做些什麼事。

In [24]:
test_yolo_output = tf.random.uniform(shape=(5, 13, 13, 255))
test_yolo_output.shape

TensorShape([5, 13, 13, 255])

In [25]:
decoder(test_yolo_output, 2).shape

output_reshape: (5, 13, 13, 3, 85)
output_reshape_txty: (5, 13, 13, 3, 2)
output_reshape_pwph: (5, 13, 13, 3, 2)
output_reshape_conf: (5, 13, 13, 3, 1)
output_reshape_class_prob: (5, 13, 13, 3, 80)
y: (13, 13)
x: (13, 13)
xy_grid: (13, 13, 2)
xy_grid: (5, 13, 13, 3, 2)
xy_grid: (5, 13, 13, 3, 2)
pred_xy: (5, 13, 13, 3, 2)
anchor box: [[116.  90.]
 [156. 198.]
 [373. 326.]]
pred_wh: (5, 13, 13, 3, 2)
pred_xywh: (5, 13, 13, 3, 4)
pred_conf: (5, 13, 13, 3, 1)
pred_prob: (5, 13, 13, 3, 80)


TensorShape([5, 13, 13, 3, 85])

參考的程式碼中還有對anchor box做縮放，猜測這是為了適應於其它大小照片所做的一個設置，為了簡單說明，我這邊就不跟著這麼處理，不過如果你的照片大小不一樣的時候就要特別注意。

<img src="https://i.imgur.com/yXo3xV3.png" width="50%" />

接下來，我們需要去計算IoU，也就是Intersection over union(IoU)，上面圖片取自吳恩達老師深度學習課程，紅框的部份是物體所在的地方，紫框則是模型推論出來的地方，我們希望它們兩個的交集要大於某個閥值。

IoU的部份目前來看有很多種的計算，這邊也給出一個[知乎的連結](https://zhuanlan.zhihu.com/p/94799295)，裡面有說明著IoU、GIoU、DIoU、CIoU。然後這個[CSDN的連結_景唯acr](https://blog.csdn.net/weixin_41735859/article/details/89288493)中的說明也不錯。

In [60]:
def bounding_box_iou(boxes1, boxes2):
    """計算兩個box的IoU
    
    boxes1, boxes2: batch_size x anchor_size x anchor_size x anchor_nums x xywh    
        xy: 中心座標
        wh: 寬高        
    """
    # 首先計算出兩個框的左上、右下點座標    
    boxes1 = tf.concat([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                        boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2 = tf.concat([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                        boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)
    # 這邊的boxes已經是左上xy、右下xy的座標
    # 座標取小、取大，目前還不確定為什麼還要做一次這樣的動作
    # 理解上，左上座標一定是比右下座標來的小，反之亦然
    boxes1 = tf.concat([tf.minimum(boxes1[..., :2], boxes1[..., 2:]),
                        tf.maximum(boxes1[..., :2], boxes1[..., 2:])], axis=-1)
    boxes2 = tf.concat([tf.minimum(boxes2[..., :2], boxes2[..., 2:]),
                        tf.maximum(boxes2[..., :2], boxes2[..., 2:])], axis=-1)
    # 各自計算面積，四個元素所代表的為：lx, ly, rx, ry
    boxes1_area = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])
    boxes2_area = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])
    
    # 準備做交集面積的計算
    # 交集面積點座標，左上取大、右下取小    
    left_up = tf.maximum(boxes1[..., :2], boxes2[..., :2])
    right_down = tf.minimum(boxes1[..., 2:], boxes2[..., 2:])
    
    # 計算交集面積的長寬
    inter_section = tf.maximum(right_down - left_up, 0.0)
    # 計算交集面積
    inter_area = inter_section[..., 0] * inter_section[..., 1]
    # 計算聯集面積
    union_area = boxes1_area + boxes2_area - inter_area
    
    iou = 1.0 * inter_area / union_area
    # 回傳IoU
    return iou, union_area

現在我們來弄個隨機資料測試看看函數執行是否正常

In [48]:
box1 = tf.random.uniform(shape=(1, 13, 13, 3, 4), minval=0, maxval=416, dtype=tf.int32)
box1 = tf.cast(box1, tf.float32)
box2 = tf.random.uniform(shape=(1, 13, 13, 3, 4), minval=0, maxval=416, dtype=tf.int32)
box2 = tf.cast(box2, tf.float32)

In [51]:
box1[0, 0, 0, 0, :], box2[0, 0, 0, 0, :]

(<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 53.,  68., 278.,  51.], dtype=float32)>,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([276., 237., 249., 311.], dtype=float32)>)

In [54]:
iou, _ = bounding_box_iou(box1, box2)

iou回傳的是batch size x anchor size x anchor size x anchor nums，因為上面的隨機範例中取的是一筆資料，所以這邊回傳的就會是這一筆資料中的13x13個grid cell中的每一個anchor boxes的iou

In [56]:
iou.shape

TensorShape([1, 13, 13, 3])

In [59]:
iou[0, 0, 0, :]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.00533298, 0.47044623, 0.        ], dtype=float32)>

IoU很明顯的就是用交集、聯集來做為一個計算比例，總面積有多少，有多少重疊，這是一個簡單的說明。而GIoU的話主要是改善當兩個框沒有重疊的時候有問題。因此在GIoU中引入一個C框，這個C框是能夠包住A、B兩框的最小框：  
$$
GIoU = IoU - \dfrac{\vert A_c - U \vert}{\vert A_c \vert}
$$

$IoU$我們知道，$A_c$指的就是C這個包住A、B兩個框的最小面積，然後$U$是A、B兩個框的聯集，用execl每個格子給個1，拉個10x10的格子來試算一下就會有感覺。

![](https://hackmd.io/_uploads/ryGDTCORq.png)

IoU減去的部份或許可以理解成C框的空白佔比，愈小很明顯的就是框的愈準，暫比愈小，假設A、B兩框直接密合，那C框大小也會等於A、B，計算出來的部份自然為0，因為$A_c-U=0$，最壞的情況就是兩個框完全沒有交集，產生的自然就會是負數(界於-1\~1之間，對比IoU是0\~1之間多了負值區間)。這時候計算它的loss，$1 - GIoU$就會變大，目標是讓loss愈小愈好，收斂效果理論上可以更好。這感覺就有點像是ReLU與Leaky ReLU之類的差異。後續有空再來讀相關論文。

In [61]:
def bounding_box_giou(boxes1, boxes2):
    """計算兩個box的GIoU
    
    boxes1, boxes2: batch_size x anchor_size x anchor_size x anchor_nums x xywh    
        xy: 中心座標
        wh: 寬高        
    """
    # 首先取得兩個框的iou與聯集面積
    iou, union_area = bounding_box_iou(boxes1, boxes2)
    
    # C框的點取得，左上取小，右下取大
    enclose_left_up = tf.minimum(boxes1[..., :2], boxes2[..., :2])
    enclose_right_down = tf.maximum(boxes1[..., 2:], boxes2[..., 2:])
    # 計算C框的長、寬
    enclose = tf.maximum(enclose_right_down - enclose_left_up, 0.0)
    # 計算C框的面積
    enclose_area = enclose[..., 0] * enclose[..., 1]
    
    giou = iou - 1.0 * (enclose_area - union_area) / enclose_area
    
    return giou

In [63]:
giou = bounding_box_giou(box1, box2)

In [65]:
giou[0, 0, 1, :]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([      inf, 0.3921479, 4.3075695], dtype=float32)>

最後需要的就是計算loss function，框的好不好(IoU)會有loss、置信度會有loss、類別也會有loss，這幾個loss加總起來就是總的loss，想當然我們希望這個loss愈小愈好。

針對loss function的部份看了一下論文，大概就只有[YOLOv1](https://hackmd.io/@shaoeChen/SyjI6W2zB/https%3A%2F%2Fhackmd.io%2F%40shaoeChen%2FHy6kUMWNI)有特別說明，下圖最上面是框的好不好的loss，中間是置信度的loss、最下面則是類別的loss：  
<img src="https://hackmd.io/_uploads/Skico6hA9.png" width="50%"/>



對於框的好不好的loss部份，程式碼中可以看的出來有一個bbox_loss_scale：  
![](https://hackmd.io/_uploads/HkoYKD90c.png)

很明顯的，框愈大的，這個scale就愈小，愈小的就愈大，這有一種正規化的感覺，關於這部份我個人有幾點想法：
1. 猜測主要是希望將我們實際拉框的資訊也列進來考慮，不然整個loss function會發現到，實際的框在這裡面是沒有角色的
2. 因為拉進實際框的考量，所以我們嚐試減少框的大小對loss的影響
3. 小框的scale拉大，我們可以想像一下，一張照片裡面可能有多個物件被框到，有的框大，有的框小，框大的好框，框小的不好處理，所以我們希望加大模型對小框的處理

下面給出一個完整的loss function，裡面加入亂七八糟的註解說明，當然在實際程式上線的時候並不會這麼做，只是在說明的時候這樣子比較能清楚知道每一段程式碼的用處。

In [77]:
def yolo_loss(output_decoder, output_raw, label, bboxes):
    """計算loss
    這邊我們假設我們的輸入就會是416
    
    output_decoder: 整理過的output
    output_raw: 原始的output
    label: 訓練資料
    bboxes: 
    """    
    # iou閥值，這部份你可以自己看是要參數化還是要做為環境變數    
    IOU_LOSS_THRESH = 0.5
    
    conv_shape = tf.shape(output_raw)
    batch_size = conv_shape[0]
    
    # 13x13, 26x26, 52x52
    anchor_size = conv_shape[1]
    input_size = 416
    
    # 這邊假設類別跟論文一樣是80，因此為5 + 80
    conv = tf.reshape(output_raw, shape=(batch_size, anchor_size, anchor_size, 3, 85))
    # 原始的置信度
    conv_raw_conf = conv[..., 4: 5]
    # 原始的類別機率
    conv_raw_prob = conv[..., 5:]
        
    # 整理過的框資訊
    pred_xywh = output_decoder[..., 0: 4]
    # 整理過的置信度
    pred_conf = output_decoder[..., 4: 5]
    
    # 實際的框資訊
    label_xywh = label[..., 0: 4]
    # 實際的框有沒有東西
    respond_bbox = label[..., 4: 5]
    # 實際的類別機率
    label_prob = label[..., 5:]
    
    # 首先計算框的好不好的loss
    # 計算預測的與實際的框之間的IoU
    giou = tf.expand_dims(bounding_box_giou(pred_xywh, label_xywh), axis=-1)       
    # input size型別轉換
    input_size = tf.cast(input_size, tf.float32)
    # 框愈大，值愈小，框愈小，直愈大，那框的大小自然對於整體的loss影響就不至於太大，
    # 要注意，這邊框的資訊是實際框，而不是anchor prior或是學習到的那個框        
    bbox_loss_scale = 2.0 - 1.0 * label_xywh[:, :, :, :, 2:3] * label_xywh[:, :, :, :, 3:4] / (input_size ** 2)
    # 很明顯的，如果實際框是有東西的，那respond_bbox就會是1
    giou_loss = respond_bbox * bbox_loss_scale * (1 - giou)
    
    # 接下來我們要做的就是計算第二部份的loss，也就是置信度的部份    
    # 找出每個grid cell中最大IoU的框
    iou = bounding_box_iou(pred_xywh[:, :, :, :, np.newaxis, :], bboxes[:, np.newaxis, np.newaxis, np.newaxis, :, :])
    max_iou = tf.expand_dims(tf.reduce_max(iou, axis=-1), axis=-1)
        
    # 前面(1.0 - respond-bbos)的部份，只要有東西的話這個結果就會是0，自然就不列入考慮
    # 是背景的，小於閥值的那一陀就是1，我們就知道這一堆就是背景
    respond_bgd = (1.0 - respond-bbos) * tf.cast(max_iou < IOU_LOSS_THRESH, tf.float32)
    
    conf_focal = tf.pow(respond_bbox - pred_conf, 2)
    conf_loss = conf_focal * (
        # 有東西的
        respond_bbox * tf.nn.sigmoid_cross_entropy_with_logits(labels=respond_bbox, logits=conv_raw_conf)
        +
        # 背景
        respond_bgd * tf.nn.sigmoid_cross_entropy_with_logits(labels=respond_bbox, logits=conv_raw_conf)
    )
    
    # 最後就是產品類別，這比較簡單
    prob_loss = respond_bbox * tf.nn.sigmoid_cross_entropy_with_logits(labels=label_prob, logits=conv_raw_prob)
    
    # 把這三個loss做一個維度的收斂，加總取平均，最終得到的就是每個樣本的loss
    # axis的概念就是，沿著那個維度做operator，那個維度就消失，
    # 所以沿著1、2、3、4，那留下的就是樣本維度
    giou_loss = tf.reduce_mean(tf.reduce_sum(giou_loss, axis=[1, 2, 3, 4]))
    conf_loss = tf.reduce_mean(tf.reduce_sum(conf_loss, axis=[1, 2, 3, 4]))
    prob_loss = tf.reduce_mean(tf.reduce_sum(prob_loss, axis=[1, 2, 3, 4]))
    
    return giou_loss, conf_loss, prob_loss
    

這個loss function很有意思，把相關可以考慮的都列進來了，下一步我們終於可以開始嚐試訓練模型了。

## 附錄

下面給出經過tf.tile處理之後的結果，較為直觀瞭解

In [21]:
tf.tile(tf.range(13, dtype=tf.int32)[:, tf.newaxis], [1, 13])

<tf.Tensor: shape=(13, 13), dtype=int32, numpy=
array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
       [ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3],
       [ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4],
       [ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5],
       [ 6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6],
       [ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7],
       [ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8],
       [ 9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9],
       [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
       [12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]], dtype=int32)>

In [23]:
tf.tile(tf.range(13, dtype=tf.int32)[tf.newaxis, :], [13, 1])

<tf.Tensor: shape=(13, 13), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]], dtype=int32)>

In [48]:
y = tf.tile(tf.range(13, dtype=tf.int32)[:, tf.newaxis], [1, 13])
x = tf.tile(tf.range(13, dtype=tf.int32)[tf.newaxis, :], [13, 1])

In [49]:
grid_xy = tf.concat([x[:, :, tf.newaxis], y[:, :, tf.newaxis]], axis=-1)
grid_xy[0]

<tf.Tensor: shape=(13, 2), dtype=int32, numpy=
array([[ 0,  0],
       [ 1,  0],
       [ 2,  0],
       [ 3,  0],
       [ 4,  0],
       [ 5,  0],
       [ 6,  0],
       [ 7,  0],
       [ 8,  0],
       [ 9,  0],
       [10,  0],
       [11,  0],
       [12,  0]], dtype=int32)>

In [50]:
# 假設batch size = 5
grid_xy = tf.tile(grid_xy[tf.newaxis, :, :, tf.newaxis, :], [5, 1, 1, 3, 1])
grid_xy.shape

TensorShape([5, 13, 13, 3, 2])

In [55]:
tf.concat([tf.concat([grid_xy, grid_xy], axis=-1), grid_xy], axis=-1).shape

TensorShape([5, 13, 13, 3, 6])